In [ ]:
import os

zip_path = "/content/MRI.zip"

if os.path.exists(zip_path):
    print("ZIP file exists.")
else:
    print("ZIP file does not exist.")

In [ ]:
import zipfile

zip_path = "/content/MRI.zip"

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        bad_file = zip_ref.testzip()
        if bad_file:
            print(f"Corrupt file found: {bad_file}")
        else:
            print("ZIP file is valid.")
except zipfile.BadZipFile:
    print("Error: The file is not a valid ZIP archive.")
except FileNotFoundError:
    print("Error: The specified ZIP file was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [ ]:
import zipfile

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    bad_file = zip_ref.testzip()
    if bad_file:
        print(f"Corrupt file found: {bad_file}")
    else:
        print("ZIP file is valid.")

In [ ]:
import tempfile
import os

extract_dir = tempfile.mkdtemp()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        try:
            zip_ref.extract(file, extract_dir)
        except Exception as e:
            print(f" Error extracting {file}: {e}")

print(f" Dataset extracted to: {extract_dir}")

In [ ]:
extract_dir = "/tmp/tmpxs769cm6"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extracted to: {extract_dir}")

In [ ]:
for root, dirs, files in os.walk(extract_dir):
    print(root, dirs, len(files), "files")

In [ ]:
def find_data_directory(temp_dir):
    """
    Locate the extracted dataset folder correctly.
    """
    possible_dirs = ['Training', 'Train', 'training']

    for dirname in possible_dirs:
        if os.path.exists(os.path.join(temp_dir, dirname)):
            return os.path.join(temp_dir, dirname)

    for item in os.listdir(temp_dir):
        item_path = os.path.join(temp_dir, item)
        if os.path.isdir(item_path):
            for dirname in possible_dirs:
                if os.path.exists(os.path.join(item_path, dirname)):
                    return os.path.join(item_path, dirname)

    return temp_dir
data_dir = find_data_directory(extract_dir)
print(f"Data directory found at: {data_dir}")

In [ ]:
import os

data_dir = "/tmp/tmpxs769cm6/Training"

for root, dirs, files in os.walk(data_dir):
    print(root, dirs, len(files), "files")

In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

IMAGE_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 10
DATASET_PATH = "/tmp/tmpxs769cm6/Training"
NUM_CATEGORIES = 4

def create_model():
    input_layer = tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    base_model = tf.keras.applications.MobileNetV2(
        input_tensor=input_layer,
        include_top=False,
        weights='imagenet'
    )

    for layer in base_model.layers[-10:]:
        layer.trainable = True

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    output_layer = tf.keras.layers.Dense(NUM_CATEGORIES, activation='softmax')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    return model, base_model

def train_model():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.3,
        brightness_range=[0.7, 1.3],
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        DATASET_PATH,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset="training"
    )

    val_generator = train_datagen.flow_from_directory(
        DATASET_PATH,
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset="validation"
    )

    model, base_model = create_model()

    model.compile(
        optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        callbacks=[
            EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
        ]
    )

    model.save('brain_tumor_detector.keras')

    final_train_acc = history.history['accuracy'][-1] * 100
    final_val_acc = history.history['val_accuracy'][-1] * 100
    print(f"\nFinal Training Accuracy: {final_train_acc:.2f}%")
    print(f"Final Validation Accuracy: {final_val_acc:.2f}%\n")

    plt.figure(figsize=(6, 4))
    plt.bar(['Training Accuracy', 'Validation Accuracy'], [final_train_acc, final_val_acc], color=['blue', 'orange'])
    plt.title('Training and Validation Accuracy')
    plt.ylabel('Accuracy (%)')
    plt.show()

    plt.figure(figsize=(6, 6))
    plt.pie([final_train_acc, final_val_acc], labels=['Training Accuracy', 'Validation Accuracy'],
            autopct='%1.1f%%', startangle=90, colors=['blue', 'orange'])
    plt.title('Accuracy Distribution')
    plt.show()

    return model, base_model

def generate_gradcam(model, img_path, class_idx=None):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img_tensor = img / 255.0
    img_tensor = np.expand_dims(img_tensor, axis=0)

    last_conv_layer = None
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer

    if last_conv_layer is None:
        raise ValueError("No convolutional layer found in the model")

    grad_model = Model(
        inputs=model.inputs,
        outputs=[last_conv_layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        if class_idx is None:
            class_idx = tf.argmax(predictions[0])

        loss = predictions[0][class_idx]

    grads = tape.gradient(loss, conv_outputs)

    conv_outputs = tf.cast(conv_outputs, dtype=tf.float32)
    grads = tf.cast(grads, dtype=tf.float32)

    cast_conv_outputs = tf.cast(conv_outputs > 0, 'float32')
    cast_grads = tf.cast(grads > 0, 'float32')
    guided_grads = cast_conv_outputs * cast_grads * grads

    pooled_grads = tf.reduce_mean(guided_grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()

    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    return superimposed_img

def main():
    model, base_model = train_model()
    img_path = '/content/brain image.jpg'
    superimposed_img = generate_gradcam(model, img_path)
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

if __name__ == "__main__":
    main()


In [1]:
!mkdir Brain-Tumor-Classification
!mkdir Brain-Tumor-Classification/src
!mkdir Brain-Tumor-Classification/notebooks
!mkdir Brain-Tumor-Classification/data
!mkdir Brain-Tumor-Classification/results


In [10]:
!cp *.ipynb Brain-Tumor-Classification/notebooks/

cp: cannot stat '*.ipynb': No such file or directory


In [3]:
!pwd

/content


In [4]:
!ls

Brain-Tumor-Classification  MRI.zip  sample_data


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!ls "/content/drive/MyDrive"


'certificates .pdf'  'Colab Notebooks'	 CV


In [9]:
!cp "/content/drive/MyDrive/Colab Notebooks/brain_tumor_classification.ipynb" Brain-Tumor-Classification/notebooks/


In [12]:
!ls "/content/drive/MyDrive/Colab Notebooks"


brain_tumor_classification.ipynb


In [13]:
!mkdir -p Brain-Tumor-Classification/notebooks
!mkdir -p Brain-Tumor-Classification/src
!mkdir -p Brain-Tumor-Classification/data
!mkdir -p Brain-Tumor-Classification/results
